# Pandas
Pandas (Python Data Analysis Library) is an extremely popular module which you'll find at the top of a huge proportion of data science notebooks. Like numpy, it's also popular enough to deserve its own import idiom...

In [ ]:
import pandas as pd
import numpy as np

To get a quick idea of the scope of pandas take a look at the autocomplete for pd.

In [ ]:
pd.

The fundamental objects in pandas are the `Series` and the `DataFrame`. These build on the idea of a numpy `ndarray`, but they add the idea of an index and they include a rich set of methods which we can use to manipulate data.

## Series
The pandas `Series` object is basically a one dimensional _indexed_ array. Schematically, they look like

| Index | Value |
|-------|-------|
| 0     |  0.12 |
| 1     |  0.24 |
| 2     |  0.36 |
| 3     |  0.48 |

Each series has an index and a value. Most of the time the index values are distinct (this isn't a firm requirement though!) but the index doesn't have to be integers. Any hashable type will do

In [ ]:
s1 = pd.Series({'one' : 1.0, 'two': 2.0, 'three': 3.0})
s1

One particularly common option is to use a timestamp as the index (don't worry about the syntax here, we'll come back to time-series later)...

In [ ]:
dti = pd.date_range('2020-06-03', periods=3, freq='h')
pd.Series(['first', 'second', 'third'], index=dti)

The next thing to notice is that the values all share the same type (e.g. `dtype: float64` for `s1`, `object` for `dti`). `pandas` will make `Series` of almost any type as long as all the rows share the same type, and it will always try to pick the most efficient implementation. For numerical values, it'll use `numpy.ndarrays`, but if it can't infer a specific type, it will fall back to a generic `object` type. It really pays (in speed) to keep an eye on the `dtype` and use a `numpy.ndarray` type where possible. For categorical values there is also a special `Categorical` `dtype` which can help later on when interfacing with `sklearn`.

### Making `Series`

You'll probably find that you want to deal with `DataFrames` more often than `Series`, but lots of operations in `panadas` return `Series` (or want them as an argument in some operation) so it's good to know a bit about them.

Like `ndarray`'s, you can initialize a `Series` from an iterable, by default it'll be given an ascending integer index

In [ ]:
sf = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0])
sf

In [ ]:
si = pd.Series(range(5))
si

In [ ]:
si.dtype

So far, this looks a lot like a numpy array (or even just a list), but we can switch the indexing to suit our needs, by explicitly passing the `index=` arguement

In [ ]:
sm = pd.Series([1., 2., 3., 4., 5.], index=['one', 'two', 'three', 'four', 'five'])
sm

### Indexing and Slicing

Square bracket notation will select by index value, this is convenient, but has ambiguities, particularly when you start manipulating `DataFrames` (see `.loc` and `.iloc` later for more flexibility). For a `Series` square brackets will try to match a row

In [ ]:
sm['three']

Now the series is starting to look more like a dictionary, in fact, that's another good way to construct `Series`

In [ ]:
sn = pd.Series({'one': 1, 'three': 3, 'two': 2, 'four': 4, 'five': 5})
sn

It even has a keys method...

In [ ]:
sn.keys()

When you look closely though, a Series has a few tricks that a dictionary doesn't...

In [ ]:
sn['three':'four']

N.B. label based indexes are _inclusive_ of the `stop` value. This is different from most other indexes you'll see in python, and can cause a little confusion. If you use numbers to do the slice, you'll get the familiar python behaviour

In [ ]:
sn[1:3]

We can operate on the entire series at once, and the usual python binary and unary operators do the right thing. e.g. Element by element comparison statements evaluate to `Boolean`s

In [ ]:
sn > 2

and if you remember the material on numpy fancy indexing, this can be very useful

In [ ]:
sn[sn > 2]

You can combine these conditions in more complicated expressions, but be careful about `and` and `&` in general you want to arrange for bitwise operations on individual elements, e.g.

In [ ]:
sn[(sn > 2) & (sn < 5)]

There are also some extra indexing methods available to you: `.loc` & `.iloc`. These let you be much more explicit about exactly what you want to return. At a very basic level, they can help you avoid label confusion...

In [ ]:
sa = pd.Series(data=['apple', 'banana', 'orange', 'pineapple'], index=[3, 2, 1, 7])

sa

Suppose we want the value in the 4th row...

In [ ]:
sa[3]

Nope, `3` was interperted as a label. There's an ambiguity because of the type of the index. Fortunately `pandas` gives us a pair of functions to avoid that problem

  * `.iloc[]`: Purely integer-location based indexing.
  * `loc[]` : Purely label based indexing.

In [ ]:
sa.loc[3]

In [ ]:
sa.iloc[3]

And both `.loc` and `.iloc` will also work with slices

In [ ]:
sa.iloc[:3:2]

Logical and fancy indexing work here too

In [ ]:
sa.iloc[[2, 1, 3]]

Fancy indexing with boolean masks is pretty common

In [ ]:
sn.loc[sn>3]

_**Exercise**: Create a series with the letters of the alphabet as values (try using `enumerate`). Use the .`sort_index` method to reverse the `Series` and slice the result to pick out the values between `t` and `k` (inclusive)_

## DataFrames

Most of the time you will be using `DataFrames` rather than `Series` (though the result of many `DataFrame` operations is a `Series`), but at a first pass it is OK to think of `DataFrames` as a bunch of `Series` stuck together with a common index. 


| Index | Value1 | Value2 | 
|-------|--------|--------|
|   0   |  0.12  |   'a'  |
|   1   |  0.24  |   'b'  |
|   2   |  0.36  |   'c'  |
|   3   |  0.48  |   'd'  |

The rules discussed above for the index stay the same but notice that we can now have different types in the various columns

In [ ]:
d1 = pd.DataFrame({'floats': sm, 'ints': sn})
d1

In [ ]:
d1.dtypes

Also, notice that when building a `DataFrame` out of multiple `Series`, pandas will try to align records based on the index values. This can be fine tuned, but the default behaviour is to fill in `NaN` or `None` values to indicate missing data. You will see something similar when we start looking at joining `DataFrames` below.

In [ ]:
pd.DataFrame({
    'count' : {'a' : 1, 'b': 6, 'c': 3},
    'uppercase': {'a': 'A', 'c' : 'C', 'd': 'D'}
})

### DataFrame Attributes

We've already seen some of the attributes of the DataFrame (column etc.) but there are quite a few available, take a look at `index`, `columns`, `shape`, `dtypes`

In [ ]:
d1.index

In [ ]:
d1.columns

In [ ]:
d1.shape

In [ ]:
d1.dtypes

There are also some useful utility functions for getting oriented with the contents of the dataframe

In [ ]:
d1.describe()

(For richer datasets, if you pass `include=all` to describe, it will also try to tell you what it can about non-numerical values)

In [ ]:
d1.info()

In [ ]:
d1.tail(n=3)

### Indexing

One important thing to note is when indexing a dataframe, the default is to give you the column (you can also use the syntax `d1.ints`)

In [ ]:
d1['ints']

If you are looking for the row, then try `.loc` with the row index value

In [ ]:
d1.loc['one']

In fact, you probably want to lean towards `.loc` when you want to subset rows and/or columns. It is much more flexible and helps pandas resolve some potential ambiguities about what you want to select. Here is a column selection...

In [ ]:
d1.loc['four':, ['ints']]

(The square brackets around `'ints'` here tell pandas I want a `DataFrame` to be returned, rather than a `Series`)

In [ ]:
d1.iloc[:2]

In [ ]:
# `.iloc will interprete both row and column specifiers as numerical positions
d1.iloc[:2, 1]

Logical/fancy indexing works with `.loc` and `.iloc` as well, but remember the parentheses or `&` will be interpreted as a bitwise operator.

In [ ]:
d1.loc[(d1['ints'] > 2) & (d1['floats'] < 4), ['ints']]

Selecting a single column will return a series, so you'll have access to Series methods

In [ ]:
type(d1['ints'])

In [ ]:
d1.loc[d1.ints.isin((1, 4))]

### Manipulating DataFrames


Certain operations can modify a `DataFrame` in place, e.g.

In [ ]:
d1.iloc[1] = (3.0, 3)
d1

In [ ]:
print(f"Before {id(d1)}")
d1['ratio'] = d1['ints'] / (2 * d1['floats'])
print(f"After {id(d1)}")
d1



And some functions/methods support the `inplace=True` keyword argument

In [ ]:
d6 = d1.copy()
d6.replace(5, 6, inplace=True)
d6

But generally pandas will return a copy of the `DataFrame` or `Series` in question. This sounds a bit wasteful, but most of the time it is better to worry about it. The main time it comes up is when you try to do something like

In [ ]:
d1['floats']['five'] = 5.1

In [ ]:
d1.loc['five', 'floats'] = 5.1

You probably intended to modify the values in place, but as the warning message said, the implementation would have made the changes to a copy of the data, leaving the original unmodified. This is such a common mistake that the warning includes a proposed fix which would do the right thing. Basically the fix lets python think about your assignment statement in one step, rather than trying to break it into two separate operations with the chained `[][]` notation. You coud use `.loc` as described above or for a single value, there is an efficient `.at` method...

In [ ]:
d1.at['five', 'floats'] = 5.0

For more general cases you can use `pd.DataFrame.loc`, (N.B. Avoid chained indexing ([][])

In [ ]:
d1.loc[:, 'floats'] += 0.1
d1

`pandas` also has facilities like `fillna` or `replace` for manipulating `DataFrames` to fill in multiple missing values using various strategies, we'll look at those later.

## DataFrame Methods

There are *lots* of methods for operating on DataFrames, have a look at the tab completion and explore the documentation for them. In particular, take a look at the help for `describe`, `head` and `tail`. These are great for orienting yourself with a new dataset

We won't have time to dive into all of the methods but we'll sample a few, then you should explore the documentation for others. Some of the most often used are

* `min`, `max`
* `mean`, `mode`, `median`
* `max`, `min`, `idxmax`, `idxmin`
* `any`, `all`
* `astype`
* `dropna`
* `sort_index`, `sort_values`
* `plot`

In most cases pandas operations will return a copy of the `DataFrame` or `Series` in question. If you want to avoid that, for simple cases, the `inplace=True` argument can help, or for more advanced cases `pd.eval` and `pd.query` can also optimize some operations. A related project called [polars](https://pola.rs/) also implements DataFrames efficiently and shares a lot with pandas.

In [ ]:
d1.min()

In [ ]:
(d1 > 3).any(axis=1)

You can coerce values to different type

In [ ]:
d1['floats'].astype(int)

And sort values (or indices)

In [ ]:
d1.sort_values('floats')

_**Exercise**: Calculate the mean value of the floats column and use it to calculate how far each float is from the mean_

The `dropna` method can come in handy sometimes. In general, pandas is very good at [handling missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html), but in some other modules will choke when they encounter a `np.NaN`

In [ ]:
d1.at['three', 'floats'] = np.nan
d1

Operating directly on the numpy array...

In [ ]:
d1['floats'].values

In [ ]:
d1['floats'].values.mean()

Operating through pandas...

In [ ]:
d1['floats'].mean()

In [ ]:
d1['floats'].dropna().values.mean()

There are also convenience methods like `.ffill` for forward or backward filling missing values etc.

## Working with External Data

Data comes in many forms from simple csv/json files, real-time APIs, structured binary files and many others. Try `pd.read_<TAB>` to see some of the `pandas` ingestion options. `read_csv` is the main workhorse for data sets which will fit on a single machine. It is way more flexible than it's name suggests (S3 buckets, https, compressed files, ...) and many of the arguments to `read_csv` will have equivalents for the other functions, so we'll start by looking at it.
    
We need a CSV to work with. The city of Vancouver has an [open data catalog](https://vancouver.ca/your-government/open-data-catalogue.aspx), which has CSV for some of the datasets. There's a dataset which lists all of the [community gardens and food trees](https://opendata.vancouver.ca/explore/dataset/community-gardens-and-food-trees/information/) maintained by the city. A copy of it is available in this directory called `CommunityGardensAndFoodTrees.csv`

In [ ]:
gardenDF = pd.read_csv(
    "https://opendata.vancouver.ca/explore/dataset/community-gardens-and-food-trees/download/?format=csv&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B",
    delimiter=';'
)
gardenDF.shape

So there are 167 rows, with 20 columns, here are the fist few rows

In [ ]:
gardenDF.head()

So we can tell things like the gardens were created (`YEAR_CREATED`), and where the are (`LATITUDE`, `LONGITUDE`), and who's responsible for them (`STEWARD_OR_MANAGING_ORGANIZATION`). 

Now it is time to clean the data. This is a hugely important step and will generally eat a lot of your time, but it is worth doing right. Having mistakes in your data can undermine everything you subsequently try to infer from it.

First let's look at the index, the default is to index by integer, but we could have picked any column instead. It looks like the first column is unique (`MAPID`) and so let's use that (chosing the index right can make your life much easier when adding data or combining multiple DataFrames). The `inplace=True` argument means modify the existing dataframe rather than returning a modified copy

In [ ]:
gardenDF.set_index('MAPID', inplace=True)

One useful trick when cleaning data is to look at the unique values in a column. You'll often catch coding mistakes or values being used as placeholders this way, e.g.

In [ ]:
gardenDF['YEAR_CREATED'].unique()

Value counts can also be useful to spot outliers

In [ ]:
gardenDF['Geo Local Area'].value_counts()

For the `YEAR_CREATED`, `Pre-2010`, `pre-1970` and `pre 2000` are kind of usless (and inconsistent!) so let's toss them

In [ ]:
for badLabel in ['Pre-2010', 'pre-1970', 'pre 2000']:
    gardenDF = gardenDF.loc[gardenDF['YEAR_CREATED'] != badLabel]

# gardenDF.query("YEAR_CREATED in ['Pre-2010', 'pre-1970', 'pre 2000']")

gardenDF['YEAR_CREATED'].unique()

There's still an `np.nan`, and the years are strings (numbers would be better, or even dates). We can take a closer look at the rows which are missing their `YEAR_CREATED` using isnull

In [ ]:
gardenDF.loc[gardenDF['YEAR_CREATED'].isnull()]

Pandas has some facilities for [dealing with missing data](https://pandas.pydata.org/docs/user_guide/missing_data.html) as well as some helpful default behaviours. Additionally, the `.isna` and `.notna` methods can help you explicitly deal with missing values during operations, e.g.

In [ ]:
missingDF = pd.DataFrame(
    np.random.randn(4, 3),
    index = ['one', 'two', 'three', 'four'],
    columns = ['a', 'b', 'c']
)
missingDF['d'] = list(np.random.randn(2)) + [np.nan] + [np.random.randn()]
missingDF

_**Exercise**:In some cases you might want to patch up missing data by combining one DataFrame with another, or by interpolating from nearby values_

  * _Use `.ffill` to fill with the value from the row above and/or below_
  * _Fill with the mean of the other row/column values_

In [ ]:
missingDF.ffill(axis=1)

In [ ]:
missingDF.fillna(missingDF.mean())

Getting back to our example, for cases where that isn't possible (like here, where I'm too lazy), you can also just throw away the rows without valid years

In [ ]:
gardenDF.dropna(axis=0, subset=['YEAR_CREATED'], inplace=True)
gardenDF

Now it might make sense to convert that column to a numerical value

In [ ]:
pd.to_numeric(gardenDF['YEAR_CREATED'])

or better yet, a DateTime object

In [ ]:
gardenDF['YEAR_CREATED'] = pd.to_datetime(gardenDF['YEAR_CREATED'])
gardenDF['NUMBER_OF_FOOD_TREES'] = pd.to_numeric(gardenDF['NUMBER_OF_FOOD_TREES'].str.replace('yes', '1'))
gardenDF['NUMBER_OF_PLOTS'] = pd.to_numeric(gardenDF['NUMBER_OF_PLOTS'])
gardenDF['STREET_DIRECTION'] = gardenDF['STREET_DIRECTION'].astype('category')
gardenDF.dtypes

Sometimes you are not so lucky and will have to parse through strings to extract the information you are looking for. The `Geom` column looks like latitude and longitude stored as json, lets parse an individual entry

In [ ]:
import json

geom = gardenDF.iloc[-1]['Geom']
json.loads(geom)

Now we have a `dict` and `coordinates` holds the longitude and latitude. We can use `pd.DataFrame.apply` to apply this transformation to all of the rows and generate new columns

In [ ]:
gardenDF['Longitude'] = gardenDF['Geom'].apply(lambda x: json.loads(x)['coordinates'][0])
gardenDF['Latitude']  = gardenDF['Geom'].apply(lambda x: json.loads(x)['coordinates'][1])
gardenDF[['Latitude', 'Longitude']]

Once you have an idea of what your data source looks like there are some arguments to `read_csv` (and related functions) which can tidy things up as they are loaded

In [ ]:
import json

def geom2lat(geom):
    if geom:
        return json.loads(geom)['coordinates'][0]
    else:
        return np.nan

    
gardenDF = pd.read_csv(
    "https://opendata.vancouver.ca/explore/dataset/community-gardens-and-food-trees/download/?format=csv&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B",
    usecols = [
        'MAPID',
        'YEAR_CREATED',
        'NAME',
        'STEWARD_OR_MANAGING_ORGANIZATION',
        'STREET_NUMBER',
        'STREET_NAME',
        'Geom',
        'Geom'
    ],
    delimiter=';',
    encoding='latin1',
    na_values={
        'YEAR_CREATED': ['Pre-2010', 'pre-1970', 'pre 2000', 'nan']
    },
    index_col='MAPID',
    parse_dates=['YEAR_CREATED'],
    converters={
        'Geom': geom2lat,
    }
)
gardenDF.head()

## Accessing Data via APIs

An API (Application Programming Interface) is a formal specification describing how systems should communicate with each other. This is an incredibly general notion which could involve information going in both directions and changes being made on both systems. We will only talk about them in the context of extracting data and massaging it into pandas, but full featured APIs could also support authentication, uploading information, telling AWS to shutdown all your ec2 instances or pretty much anything else you could think of.

APIs allow applications (your code) to interact with other applications (someone elses' code) and they're particularly useful where the incoming data is event based or otherwise frequently updated. They also turn up in situations where the entire data set is very large, but individual requests will only need small slices; applications can request the information they need on demand without the source having to pre-prepare all the possible variations for download.

Good APIs are versioned (or at the very least stable), well documented and they often implement a pattern called [REST](https://en.wikipedia.org/wiki/Representational_state_transfer). This adds a layer of formality and standardization to the API which which helps to make them more predictable and easier to develop against. Most of the API's I can think of operate over HTTP and pass information back and forth in the [json](https://en.wikipedia.org/wiki/JSON) format.


For popular APIs (e.g. [twitter](https://developer.twitter.com/en/docs/twitter-api)) you might find that someone has already written a python wrapper for the API (e.g [python-twitter](https://python-twitter.readthedocs.io/en/latest/)). This is usually the best case scenario because it means someone else has done the hard work of reading the API specification for you. In other cases, you might only have access to the API spec or maybe a console to help you build queries. If you're very unlucky, the API will be undocumented, the silver lining in this case is these are usually where the really juicy stuff is stored!

* [Vancouver Open Data Portal](https://opendata.vancouver.ca/api/v1/console/datasets/1.0/search/) has a console to help you build queries
* [Canada Open Data Portal](https://open.canada.ca/en/working-data#toc93c) has instructions on interfacing with python
* [Stats Canada API](https://www.statcan.gc.ca/eng/developers/wds/user-guide#a12-3) See also [this post](https://towardsdatascience.com/how-to-collect-data-from-statistics-canada-using-python-db8a81ce6475) on using that portal
* [Twitter API](https://developer.twitter.com/en/docs)
* [Open Weather Map](https://openweathermap.org/api)
* [NASA](https://api.nasa.gov/) imagery, data, etc.
* [GitHub](https://docs.github.com/en/rest)

If there isn't a pre-packaged module to take care of your needs the [requests module](https://docs.python-requests.org/en/master/) is usually your best option. Here is a quick example extracting the part of the gardens dataset from the vancouver data portal

In [ ]:
import requests

from urllib.parse import urlencode, urljoin

base_url = 'https://opendata.vancouver.ca/api/v2/'
catalog_item = 'catalog/datasets/council-voting-records/records'
headers = {
 'Content-Type': 'application/json; charset=utf-8'
}


params = {
    'dataset_id' : 'council-voting-records',
    'limit'      : 10,
    'where'      : 'meeting_type = "Council"',
}


r = requests.get(urljoin(base_url, catalog_item) , params=params)
r.status_code

In [ ]:
r.headers['content-type']

In [ ]:
r.json();

If you look up these params in the the [API documentation](https://opendata.vancouver.ca/api/v2/console#!/dataset/getRecords) you'll see that we are filtering for records where `meeting_type` is `Council` and we are asking for a `limit` of 10 records (which is actually the default value of `limit`). But there are more records in total which match that filter (look at the first line of `r.json()`. If we want them all, we have to access them in pages. Before we do that though, we should think about rate limits. Almost every API will implement some form of rate limit and the Vancouver Open Data (as well as a lot of other places) will tell you what your current rate limit status is in the response headers.

In [ ]:
{k:v for k, v in dict(r.headers).items() if k.startswith('X-Rate')}

[Their documentation](https://help.opendatasoft.com/apis/ods-search-v1/#quotas) has information on these values. In general, registered users will be given more generous limits. Often this means registering for some kind of token and including it in your requests, we will do this below, but with one **major** caveat. The token is usually equivalent to a password and you should treat it the same way. Don't share it with people, be careful where you use it (including jupyter notebook output cells!), and make sure you don't check it into version control!

In [ ]:
import os
from getpass import getpass

In [ ]:
os.environ['VAN_API_KEY'] = getpass('API Key')

In [ ]:
params['apikey'] = os.environ['VAN_API_KEY']
r = requests.get(base_url, params=params)

In [ ]:
{k:v for k, v in dict(r.headers).items() if k.startswith('X-Rate')}

With our new rate limit we can make multiple requests and get all of the data we wanted. Even so it is worth limiting the impact we will have on their API, there are modules like [ratelimit](https://pypi.org/project/ratelimit/) which can do this systematically, but we will just add a manual delay.

In [ ]:
import time
from urllib.parse import urlencode, urljoin

records = []
params['rows'] = 50

url  = f"{urljoin(base_url, catalog_item)}?{urlencode(params)}"
json = ''
while len(records) < 250:
    time.sleep(2)
    
    r = requests.get(url)
    urls = {x['rel']:x['href'] for x in r.json()['links']}

    records.extend(r.json()['records'])
    
    if urls['self'] == urls['last']:
        break
    else:
        url = urls['next']
        params = {}

In [ ]:
records[-1];

So we have a list of these json objects, the important stuff is in the 'record' key, under 'fields'

In [ ]:
votes = pd.DataFrame([chunk['record']['fields'] for chunk in records]).set_index('vote_detail_id')
votes.head()

In [ ]:
votes.info()

In [ ]:
#[votes[col] = votes[' for col in ['meeting_type', 'vote', 'decision']]
votes = votes.astype({
    'meeting_type': 'category',
    'vote': 'category',
    'decision': 'category',
})
votes['vote_start_date_time'] = pd.to_datetime(votes.vote_start_date_time)
votes['vote_date'] = pd.to_datetime(votes.vote_date)
votes.decision.value_counts().plot(kind='pie')

## TimeSeries

We've already talked a bit about time and date handling, but the author of pandas wrote it to deal with time series data so it really excels here. The main objects to be aware of are

  * **Time Stamps**: Specific points in time usually recorded to the second or nanosecond
  * **Time Periods**: Specific blocks of (usually contiguous) time (e.g. months in a year)
  * **Time Deltas**: These types lets you do arithmetic on the above objects

and their associated indices. We need some dates to play with. There's a convenience function called `to_datetime` which can convert many "human readable" dates to a pd.Timestamp object

In [ ]:
from datetime import datetime, UTC

moonwalk = pd.to_datetime('July 20, 1969, 20:17 UTC')
moonwalk

Timestamps have attributes which let you extract days, year, etc. Normally these will be reported as numbers, but the strftime method supports the usual format specifiers (The correspond with the libc specifiers, here's a reference http://strftime.org/)

In [ ]:
moonwalk.year

In [ ]:
print(f"The moon walk took place on a {moonwalk.strftime('%A')}")

In [ ]:
pd.to_datetime(datetime.now(UTC), utc=True)

Let's look at another sample dataset. This time, it is a record of historical flight data. It contains various columns, we will look at

  * `activity_period`: The date for the record in the format yyyymm
  * `passenger_count`
  
We can parse the date into a DateTime with `pd.to_datetime` then set the result as the index. We can then slice dates and resample

In [ ]:
r = requests.get('https://data.sfgov.org/resource/rkru-6vcg?$limit=10000')

flightsDF = pd.DataFrame(r.json())
flightsDF.head()

In [ ]:
flightsDF['passenger_count'] = flightsDF['passenger_count'].astype(int)
flightsDF['date'] = pd.to_datetime(flightsDF['activity_period'], format='%Y%m')

flightsDF.set_index('date', inplace=True)
flightsDF.head()

The raw data is a bit granular so we'll resample, (we'll discuss resampling more below)

In [ ]:
monthlyFlightsDF = flightsDF[['passenger_count']].resample('ME').sum()
monthlyFlightsDF.head()

In [ ]:
monthlyFlightsDF.loc['2006-01-01':, 'passenger_count'].plot(kind='bar')

Notice that "natural" indexing worked straight away. This is very handy!

Timestamp objects can also deal with arithmetic

In [ ]:
flightsDF.index[-1] - flightsDF.index[0]

In [ ]:
flightsDF['passenger_count'].resample('YE').mean().plot(kind='bar')

Again, the `value_counts` method can be very useful to get an idea of how categorical values are distributed...

In [ ]:
flightsDF['geo_region'].value_counts()

For many of the algorithms in scikit learn, we need to convert categorical variables into numerical values. The simplest case is [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) which Pandas can do for us with the `.get_dummies` method (sklearn also has it's own methods, but it's handy to be able to do this in general).

In [ ]:
flightsDF.columns

In [ ]:
airlines_one_hot = pd.get_dummies(flightsDF['operating_airline_iata_code'])
flightsDF = flightsDF.drop([
    'operating_airline_iata_code',
    'operating_airline',
    'published_airline',
    'published_airline_iata_code',
], axis = 1
)
flightsDF.join(airlines_one_hot).head()

_**Exercise**: Try resampling to find the total passenger count each quarter_

In [ ]:
flightsDF['passenger_count'].resample('3ME').sum()

## Grouping, Joining, Concatenating

When working with a new dataset, I recommend trying to stuff as much as you can into a single `DataFrame` to try to help find the basic patterns, but sooner or later you will want to do aggregate operations within a `DataFrame` (e.g. group together all of the rows by year and show the mean value of some other column) or combine two or more `DataFrame`s. 

This data set has some old information about car engine efficency

In [ ]:
carsDF = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
)
carsDF.head()

In [ ]:
carsDF.cylinders.value_counts()

### Groupby
Let's group things by `number_of_cylinders` and see how that affects mpg...

In [ ]:
carsDF['mpg'].mean()

Calling `groupby` on it's own will give you a `DataFrameGroupBy` object. Informally you can think of this as an iterator for "sub"-`Dataframe`s or slices of your original `DataFrame`. You're expected to perform aggregate operations (e.g. `sum`, `mean` or generic `.apply` methods) on these groups to actually see some results.

In [ ]:
carsDFbyCylinders = carsDF.groupby('cylinders')
carsDFbyCylinders

In [ ]:
carsDFbyCylinders.describe()

The typical aggregate operations are things like

  * mean()
  * sum()
  * median()
  * min()/max()
  * count()
  * nunique()
  * size()

In [ ]:
carsDFbyCylinders.mean(numeric_only=True)

The `agg` method lets you apply arbitrary aggregate functions as well, e.g.

In [ ]:
carsDFbyCylinders['mpg'].agg('std')

_**Exercise**: Try grouping my model year and looking at the median value for each group._

**N.B.** Reading this back, I realized there are a couple of ways to interprete this exercise, you could either start again and group _only_ by year, or you could group within our existing groupby. In the second case you are grouping by number of `cylinders`, then by `model_year`, then applying some aggregate functions

In [ ]:
carsDFbyCylinders['model_year'].agg(['median', 'mean'])

The object returned by `groupby` is an iterator, so...

In [ ]:
for vroom, group in carsDFbyCylinders:
    print(f"There are {group.shape[0]} cars with {vroom} cylinders")

How are there possibly 4 cars with 3 cylinders?!

In [ ]:
carsDFbyCylinders.get_group(3)

We can also apply multiple operations at the same time. The `.agg()` method can take a list of the operations you want to perform (e.g. ["max", "min"])

In [ ]:
carsDFbyCylinders['mpg'].agg(["min","max","mean"])

We can filter based on our groups. This isn't the recommended way of doing the following, but let's manually group the cars by cylinder count, then find any particular cars that are more than 1.5 times the mean mpg of their group.

In [ ]:
carsDF[
    carsDFbyCylinders.apply(lambda x: x.mpg > 1.5 * x.mpg.mean()).reset_index(level=0).mpg
]


There's actually a transform method lets you perform a group operation then use the results to update the rows. For example, we could calculate mean values for our groups, then look at how individual cars perform relative to that

In [ ]:
means = carsDFbyCylinders.mpg.transform('mean')
carsDF[carsDF.mpg > 1.5 * means]

We could also look at the best and worst performing car relative to the group means, `idxmin` and `idxmax` are to pandas what `argmin` and `argmax` are to numpy (see also `nlargest`).

In [ ]:
carsDF['mpg_mean'] = carsDF['mpg'] - means
carsDF.loc[carsDF.groupby('cylinders')['mpg_mean'].idxmax()]

In [ ]:
carsDF.loc[carsDF.groupby('cylinders')['mpg_mean'].idxmin()]

You can also group by multiple conditions, but you'll get a hierarchical index as the result

In [ ]:
xCars = carsDF.groupby(['origin', 'cylinders']).mean(numeric_only=True)
xCars

In [ ]:
xCars.loc[('japan', 4)]

We won't cover Hierarchical indices, so for now we will just flatten the result by resetting the index

In [ ]:
carsDF.groupby(['origin', 'cylinders'])['mpg'].mean().reset_index()

N.B. `.transform` and `.apply` exist as methods of both ordinary DataFrames and `DataFrameGroupBy` objects, depending on what you want to do.

## Concatenate & Join

There are a handful of functions which handle concatenation. The main ones are `pd.concat`, `pd.join` and `pd.merge` and there is some overlap in exactly what they do so. You can combine either `Series` and `DataFrames` but we'll jump straight to `DataFrames`,

In [ ]:
s1 = pd.DataFrame(
    {
        'upper': ['A', 'B', 'C'], 
        'lower': ['a', 'b', 'c']
    }, 
    columns=['upper', 'lower'], 
    index=[1,2,3]
)

s2 = pd.DataFrame(
    {
        'upper': ['D', 'E', 'F'],
        'lower': ['d', 'e', 'f']
    }, 
    columns=['upper', 'lower'],
    index=[1, 2, 3]
)
s2

`pd.join` will combine by aligning on the index, but expects unique column names, we can use the suffix keyword to make our columns unique

In [ ]:
s1.join(s2, lsuffix='_l', rsuffix='_r')

With concat we can combine rows (we'll reindex to get unique index values in the result)

In [ ]:
pd.concat([s1,s2]).reset_index()

Notice that we enclosed the things we want to join as some sort of iterable (a `list` here).

`concat` can also combine columns

In [ ]:
pd.concat([s1, s2], axis='columns')

`pd.concat` will accept duplicate indices, but normally that indicates a problem with the data normalization. `concat` has a `verify_index` argument which can check for these problems and you can specify what you want to do with duplicates manually.

concat will often result in `NaN`s because some columns might not exist in both/all frames.

In [ ]:
d4 = pd.DataFrame({'fruit': ['apple', 'orange'], 'veg': ['brocolli', 'carrot'], 'tree': ['cedar', 'alder']})
d5 = pd.DataFrame({'veg': ['onion', 'potato'], 'fruit': ['banana','grape']})
pd.concat([d4,d5], sort=False, ignore_index=True)

Since the index wasn't important here, I threw it away and just accepted a new one.

In the general case joining DataFrames can get complex. The concat method can take a `join` keyword to specify a database like join stragegy (inner or outer), but `pd.merge` is a bit more flexible. It implements the usual relations

  * one-to-one (similar to a concat)
  * many-to-one
  * many-to-many

and lets us join based on column value(s). Here is an example of a many-to-one merge

In [ ]:
adf1=pd.DataFrame({
    'class': ['insect', 'spider'], 
    'legs': [6, 8]}
)

adf2=pd.DataFrame({
    'name': ['molly', 'anna', 'stephen', 'mica'], 
    'class': ['insect','insect','spider','insect']}
)                     

print(adf1); print(adf2)

In [ ]:
pd.merge(adf1, adf2)

The _many-to-one_ is many different rows in adf2 being mapped to a single row in adf1 (insects). `pd.merge` also accepts a selection of keyword arguments so you can manually specify which columns to join, patch up name differences etc.

## Extra

Here is a dataset on UFO observations for the US and Canada, read it in and try to extract the following information

* _How many from each country?_
* _States with the most observations_
* _What is the most common shape observed in each country/state?_
* _Find the most recent report from each province in canada_
* _Look at the number of observations in California by month_
* _For each state find the ufo hotspot with a `.pivot table`_
  * _Since _lat_ and _lng_ are just coordinates, try plotting the hot spots_

In [ ]:
from collections import defaultdict

ufoDF = pd.read_csv(
    'https://m2pi.syzygy.ca/data/UFOs_coord.csv',
    names=['Date', 'Country', 'City', 'State', 'Shape', 'Summary', 'lat', 'lng'],
    skiprows=[0],
    encoding='latin1',
)
d1 = pd.to_datetime(ufoDF.Date.str.replace('.', ':', regex=False), format='%m/%d/%y %H:%M', errors='coerce')
d2 = pd.to_datetime(ufoDF.Date.str.replace('.', ':', regex=False), format='%m/%d/%y %H:%M:%S', errors='coerce')
d3 = pd.to_datetime(ufoDF.Date.str.replace('.', ':', regex=False), format='%Y-%m-%d %H:%M:%S', errors='coerce')
d4 = pd.to_datetime(ufoDF.Date.str.replace('.', ':', regex=False), format='%m/%d/%y', errors='coerce')
ufoDF.loc[:, 'Date'] = pd.to_datetime(d1.fillna(d2).fillna(d3).fillna(d4))
ufoDF['Date'] = ufoDF['Date'].astype('datetime64[ns]')
ufoDF.head()

How many from each country?

States with the most observations?

What is the most common shape observed in each country/state?

_Find the most recent report from each province in canada_

_Look at the number of observations in California by month_

In [ ]:
ufoDF['Month'] = ufoDF.Date.dt.month

* _For each state find the ufo hotspot with a `.pivot table`_
  * _Since _lat_ and _lng_ are just coordinates, try plotting the hot spots_

## Other bits and pieces...

In this section we'll just list some commonly used functions which didn't fit elsewhere, but you will probably see

* pd.sample: select random rows
* pd.cut: Bin data
* pd.corr: Calculate a correlation matrix
* pd.rolling: Rolling averages

In [ ]:
ufoDF.sample(5)

In [ ]:
pd.cut(ufoDF['lat'], 10).value_counts()

In [ ]:
ufoDF.loc[:, ['lat', 'lng']].corr()

In [ ]:
ufoDF['Date'].notna().sort_values().rolling(5).mean()

In [ ]:
pd.Series(1, index=ufoDF['Date']).sort_index().rolling('30d').sum().plot()